# SQL Ingest - Ingest data using SQL query 

In [ ]:
# nuclio: ignore
import nuclio

In [ ]:
%nuclio config kind = "job"
%nuclio config spec.build.baseImage = "mlrun/mlrun"

In [ ]:
%%nuclio cmd -c
pip install --no-cache-dir git+https://github.com/v3io/PyHive.git@v0.6.999 
pip install sqlalchemy==1.3.11
pip install PyMySQL==0.9.3

In [ ]:
import pandas as pd
import pyhive
from sqlalchemy.engine import create_engine
from mlrun.execution import MLClientCtx


def get_big_dataset_meta(
    context: MLClientCtx,
    database_url: str,
    table_name: str,
) -> None:

    """
    Update big dataset object attributes/metadata by using presto engine

    this method will edit or add metadata to a dataset object
    
    :param sql_query:         the sql query used to retrieve the data
    :param database_url:      database connection URL
    :param table_name:        table name to query
    """

    engine = create_engine(database_url)

    preview_sql = """SELECT * FROM {} LIMIT 100""".format(table_name)
    header_schema_sql = """SHOW COLUMNS FROM {}""".format(table_name)

    preview = pd.read_sql(preview_sql, engine)
    header_schema = pd.read_sql(header_schema_sql, engine)
    #stats = pd.read_sql(stats_sql, engine)

In [ ]:
# nuclio: end-code

### mlconfig

In [ ]:
from mlrun import mlconf
import os
mlconf.dbpath = mlconf.dbpath or 'http://mlrun-api:8080'
mlconf.artifact_path = mlconf.artifact_path or f'{os.environ["HOME"]}/artifacts'


### Save function

In [ ]:
def mount_secret(
    secret_name, volume_mount_path, volume_name='secret', items=None
):
    def _mount_secret(task):
        from kubernetes import client as k8s_client
        vol = k8s_client.V1SecretVolumeSource(secret_name=secret_name, items=items)
        return task.add_volume(
            k8s_client.V1Volume(name=volume_name, secret=vol)
        ).add_volume_mount(
            k8s_client.V1VolumeMount(mount_path=volume_mount_path, name=volume_name)
        )
    return _mount_secret

In [ ]:
from mlrun import code_to_function, NewTask
import os

fn = code_to_function(name="sql_to_file",
                      handler="sql_to_file",
                      description="SQL To File - Ingest data using SQL query",
                      categories=["data-prep"],
                      labels={"author": "adih"})

if "V3IO_ACCESS_KEY" in list(os.environ):
    fn.apply(mount_secret(secret_name='presto-tls',
                        volume_mount_path= '/var/run/iguazio/secrets/'))


### Build the image

In [ ]:
fn.deploy()

In [ ]:
fn.export('function.yaml')

## Test

### Reading from a public MySQL DB

In [ ]:
mysql_url = 'mysql+pymysql://rfamro@mysql-rfam-public.ebi.ac.uk:4497/Rfam'
mysql_query = 'select rfam_acc,rfam_id,auto_wiki,description,author,seed_source FROM family'

In [ ]:
from mlrun import NewTask, run_local

sql_task = NewTask(name='sql',
                   handler=sql_to_file,
                   params={'sql_query': mysql_query,
                           'database_url': mysql_url})


In [ ]:
sql_func = run_local(sql_task)

#### Run it on a cluster

In [ ]:
fn.run(sql_task)

### SQL query from Iguazio Key Value via Presto

You need to create a table and set the sql_table path accordingly. <br>
you can find an example of creating such table in https://github.com/v3io/tutorials/blob/master/data-ingestion-and-preparation/basic-data-ingestion-and-preparation.ipynb

In [ ]:
# nuclio: ignore
import os
sql_table = os.path.join('v3io.users."'+str(os.getenv('V3IO_USERNAME'))+'/examples/stocks_tab"')
sql_query_string = 'select * from '+sql_table+""

In [ ]:
%sql select * from $sql_table limit 10

In [ ]:
sql_task = NewTask(name='sql', 
                   handler=sql_to_file,
                   params={'sql_query': sql_query_string,
                          'database_url': os.getenv('DATABASE_URL')}
                          )


In [ ]:
sql_func = run_local(sql_task)

In [ ]:
fn.run(sql_task)